In [28]:
import os
import sys
import torch
import copy
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

# navigates to the main project folder
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), "../"))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.behavior_cloning import *
from src.utils.data_loading import *
print(project_root)

d:\University\5th_Semester\Practical_Work_in_AI\Offline_RL_PW


<h2> Loading Data </h2>

In [18]:
rb_observations, rb_next_observations, rb_actions, rb_rewards, rb_dones = load_data()
fp_observations, fp_next_observations, fp_actions, fp_rewards, fp_dones = load_data('../data/final_policy.npz')
rb_df = load_data_as_df(rb_observations, rb_next_observations, rb_actions, rb_rewards, rb_dones)
fp_df = load_data_as_df(fp_observations, fp_next_observations, fp_actions, fp_rewards, fp_dones)

<h2> Implementations </h2>

<h2> Testing </h2>

In [23]:
train_loader, test_loader, valid_loader = get_BC_data_loaders(observations=rb_observations[0:1000], actions=rb_actions.flatten()[0:1000])